# Pràctica 4

In [8]:
from datasets import load_dataset

dataset = load_dataset("projecte-aina/catalan_general_crawling")

In [9]:
dataset = dataset['train']

# i want to see the first 10 examples
for i in range(10):
    print(dataset[i])
    print()

{'text': 'Reduïu els costos dels processos administratius al vostre organisme públic\nEviteu els desplaçaments i pèrdua de temps als ciutadans en les seves gestions\nOferiu una administració més transparent a ciutadans i empreses\nEns grans i petits experimenten aquesta transformació amb èxit, gràcies al suport de l\'AOC\nDepartament de Sistemes d\'Informació i Processos\n" Via Oberta ens ha permès fer efectiu el dret dels ciutadans a no aportar documents, eliminant paper i simplificant procediments"\n" e.FACT proporciona informació indispensable per a la realització de les auditories del registre comptable de factures de les Administracions Públiques Catalanes"\nCoordinador del departament d\'Informàtica\n"El servei VIA OBERTA és el que ha aportat majors avantatges per als ciutadans"\n"Amb l\' e-NOTUM hem escurçat els procediments en 12 dies, quasi un 40% menys!"\nCoordinadora d\'organització de persones i e-administració\n" Via Oberta ofereix millores per als ciutadans al no haver d\

Obtenim diferents mides de datasets

In [10]:
# i want to get the first 100 MB of the dataset
size = 0
for i in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 100000000:
        print(i)
        break

dataset_100MB = dataset.select(list(range(i)))


# i want to get the first 500 MB of the dataset
size = 0
for j in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 500000000:
        print(j)
        break

dataset_500MB = dataset.select(list(range(j)))


# i want to get the first 1 GB of the dataset
size = 0
for g in range(len(dataset)):
    size += len(dataset[i]['text'])
    if size > 1000000000:
        print(g)
        break

dataset_1GB = dataset.select(list(range(g)))

57333
369003
738007


## Preprocessament

In [11]:
import gensim
from gensim.models import Word2Vec
import re

# Funció per netejar i tokenitzar el text
def preprocess_text(text):
    # Neteja el text: treu caràcters no desitjats, converteix a minúscules, etc.
    text = re.sub(r'\W+', ' ', text)  # Substitueix caràcters no alfanumèrics per espais
    text = text.lower()  # Converteix a minúscules
    words = text.split()  # Tokenitza
    return words

# Carrega el fitxer de text i processa'l
corpus_100MB = []
for line in dataset_100MB:
    words = preprocess_text(line['text'])
    if words:  # Assegura't que la línia no està buida
        corpus_100MB.append(words)

# Comprova algunes línies per assegurar-te que tot està bé
print(corpus_100MB[:2])


[['reduïu', 'els', 'costos', 'dels', 'processos', 'administratius', 'al', 'vostre', 'organisme', 'públic', 'eviteu', 'els', 'desplaçaments', 'i', 'pèrdua', 'de', 'temps', 'als', 'ciutadans', 'en', 'les', 'seves', 'gestions', 'oferiu', 'una', 'administració', 'més', 'transparent', 'a', 'ciutadans', 'i', 'empreses', 'ens', 'grans', 'i', 'petits', 'experimenten', 'aquesta', 'transformació', 'amb', 'èxit', 'gràcies', 'al', 'suport', 'de', 'l', 'aoc', 'departament', 'de', 'sistemes', 'd', 'informació', 'i', 'processos', 'via', 'oberta', 'ens', 'ha', 'permès', 'fer', 'efectiu', 'el', 'dret', 'dels', 'ciutadans', 'a', 'no', 'aportar', 'documents', 'eliminant', 'paper', 'i', 'simplificant', 'procediments', 'e', 'fact', 'proporciona', 'informació', 'indispensable', 'per', 'a', 'la', 'realització', 'de', 'les', 'auditories', 'del', 'registre', 'comptable', 'de', 'factures', 'de', 'les', 'administracions', 'públiques', 'catalanes', 'coordinador', 'del', 'departament', 'd', 'informàtica', 'el', 's

## Entrenament model de similitud de Text Semàntic

In [12]:
from datasets import load_dataset

text_semantic = load_dataset("projecte-aina/sts-ca")

c:\Users\Usuario\.pyenv\pyenv-win\versions\3.11.0\Lib\site-packages\datasets\load.py:1486: FutureWarning: The repository for projecte-aina/sts-ca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/sts-ca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating train split: 2073 examples [00:00, 40970.85 examples/s]
Generating validation split: 500 examples [00:00, 83906.22 examples/s]
Generating test split: 500 examples [00:00, 62495.22 examples/s]


In [27]:
from gensim.models import word2vec
model_100MB = word2vec.Word2Vec(corpus_100MB, vector_size=100, window=5, min_count=10, workers=4, epochs=25)
model_100MB.save("model_100MB")
# Obtenir un word-vector

In [31]:
model_100MB = Word2Vec.load("model_100MB")

In [32]:
# Exemple: trobar paraules similars
similar_words = model_100MB.wv.most_similar('paraula')
print(similar_words)

[('frase', 0.7631105184555054), ('grafia', 0.6595815420150757), ('lletra', 0.6338362693786621), ('locució', 0.5864616632461548), ('recomanació', 0.5816650390625), ('conjunció', 0.5588663220405579), ('pronúncia', 0.5564224720001221), ('idea', 0.5528774261474609), ('cançó', 0.5501916408538818), ('afirmació', 0.5485555529594421)]


In [17]:
text_semantic_train = text_semantic['train']

In [26]:
text_semantic_train[0]

{'sentence1': 'Atorga per primer cop les mencions Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència Universitària',
 'sentence2': 'Creen la menció M. Encarna Sanahuja a la inclusió de la perspectiva de gènere en docència universitària',
 'label': 3.5}

In [36]:
# Preprcessament del text_semantic
corpus_semantic = []
semantic_score = []
for line in text_semantic_train:
    frase1 = preprocess_text(line['sentence1'])
    frase2 = preprocess_text(line['sentence2'])

    corpus_semantic.append((frase1, frase2))
    semantic_score.append(line['label'])

### Model Baseline

In [57]:
import tensorflow as tf

def build_and_compile_model(input_length: int = 100, hidden_size: int = 64, dictionary_size: int = 1000, embedding_size: int = 16) -> tf.keras.Model:
    
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32), tf.keras.Input((input_length, ), dtype=tf.int32)
    
    # Define Layers
    embedding = tf.keras.layers.Embedding(
        dictionary_size, embedding_size, input_length=input_length, mask_zero=True)
    
    lstm = tf.keras.layers.LSTM(hidden_size)
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')
    output = tf.keras.layers.Dense(1)
    
    # Pass through the layers
    _embedded_1, _embedded_2 = embedding(input_1), embedding(input_2)
    _lstm_1, _lstm_2 = lstm(_embedded_1), lstm(_embedded_2)
    
    _concatenated = concatenate([_lstm_1, _lstm_2])
    _hidden_output = hidden(_concatenated)
    _output = output(_hidden_output)
    
    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=_output)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.01))
    return model

# Exemple d'ús del model
baseline_model = build_and_compile_model()
baseline_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 100, 16)              16000     ['input_3[0][0]',             
                                                                     'input_4[0][0]']             
                                                                                                  
 lstm_1 (LSTM)               (None, 64)                   20736     ['embedding_1[0][0]',   

### 2. Models Word2Vec pre-entrenats

In [38]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Word2Vec + Mean 
def compute_mean_embedding(text, model, embedding_dim):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    vectors = [model[word] for word in text if word in model]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(embedding_dim)
    return mean_vector


# Word2Vec + Mean Ponderada
def compute_weighted_mean_embedding(text, model, word2tfidf, embedding_dim):
    """
    Utilitza una ponderació per a cada vector de paraula, com ara la freqüència 
    inversa del document (TF-IDF), per calcular una mitjana ponderada dels vectors.
    """
    vectors = [model[word] * word2tfidf[word] for word in text if word in model and word in word2tfidf]
    if vectors:
        weighted_mean_vector = np.mean(vectors, axis=0)
    else:
        weighted_mean_vector = np.zeros(embedding_dim)
    return weighted_mean_vector

In [39]:
# Passar les frases de corpus_semantic a vectors
for i, (frase1, frase2) in enumerate(corpus_semantic):
    corpus_semantic[i] = (compute_mean_embedding(frase1, model_100MB.wv, 100), 
                          compute_mean_embedding(frase2, model_100MB.wv, 100))

In [64]:
#### Entrenament de la xarxa amb els vectors de les frases ####

# Convertir les llistes de tuples a arrays de NumPy
vectors_1 = np.array([pair[0] for pair in corpus_semantic])
vectors_2 = np.array([pair[1] for pair in corpus_semantic])

# Comprovar si hi ha valors fora de rang i substituir-los
vectors_1 = np.clip(vectors_1, 0, 999)
vectors_2 = np.clip(vectors_2, 0, 999)

# Convertir les llistes de similituds a un array de NumPy
semantic_score = np.array(semantic_score)

baseline_model.fit(
    [vectors_1, vectors_2], 
    semantic_score, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2
)

Epoch 1/50
52/52 [==============================] - 1s 27ms/step - loss: 0.6674 - val_loss: 0.6320
Epoch 2/50
52/52 [==============================] - 1s 27ms/step - loss: 0.6678 - val_loss: 0.6326
Epoch 3/50
52/52 [==============================] - 1s 26ms/step - loss: 0.6668 - val_loss: 0.6330
Epoch 4/50
52/52 [==============================] - 1s 27ms/step - loss: 0.6635 - val_loss: 0.6363
Epoch 5/50
52/52 [==============================] - 1s 26ms/step - loss: 0.6669 - val_loss: 0.6302
Epoch 6/50
52/52 [==============================] - 1s 26ms/step - loss: 0.6712 - val_loss: 0.6287
Epoch 7/50
52/52 [==============================] - 1s 26ms/step - loss: 0.6654 - val_loss: 0.6287
Epoch 8/50
52/52 [==============================] - 1s 27ms/step - loss: 0.6723 - val_loss: 0.6292
Epoch 9/50
52/52 [==============================] - 1s 26ms/step - loss: 0.6707 - val_loss: 0.6297
Epoch 10/50
52/52 [==============================] - 1s 27ms/step - loss: 0.6664 - val_loss: 0.6286
Epoch 11/

### Prova

In [36]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

# Defineix funcions per calcular els embeddings de frases
def compute_mean_embedding(text, model, embedding_dim):
    """
    Agafa els vectors d'embeddings de cada paraula en una frase o document i 
    calcula la mitjana dels vectors per obtenir una única representació vectorial 
    per a la frase o document.
    """
    vectors = [model[word] for word in text if word in model]
    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(embedding_dim)
    return mean_vector

def compute_weighted_mean_embedding(text, model, word2tfidf, embedding_dim):
    """
    Utilitza una ponderació per a cada vector de paraula, com ara la freqüència 
    inversa del document (TF-IDF), per calcular una mitjana ponderada dels vectors.
    """
    vectors = [model[word] * word2tfidf[word] for word in text if word in model and word in word2tfidf]
    if vectors:
        weighted_mean_vector = np.mean(vectors, axis=0)
    else:
        weighted_mean_vector = np.zeros(embedding_dim)
    return weighted_mean_vector

In [37]:
# Exemple de corpus
corpus = ["Aquest és un text de exemple", "Aquí hi ha un altre text"]

# Prepara els vectors TF-IDF
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(corpus)
word2tfidf = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

In [38]:
# Exemple d'ús amb embeddings pre-entrenats (Word2Vec)
from gensim.models import KeyedVectors
import tensorflow as tf

# Carrega el model pre-entrenat (substitueix 'path_to_model' amb la ruta real)
# word2vec_model = KeyedVectors.load_word2vec_format('path_to_model', binary=True)

# Defineix la longitud de les seqüències i la dimensió dels embeddings
input_length = 10
embedding_dim = model_100MB.vector_size

# Prepara les dades d'entrenament
X_train1 = np.array([compute_mean_embedding(text.split(), model_100MB.wv, embedding_dim) for text in corpus])
X_train2 = np.array([compute_weighted_mean_embedding(text.split(), model_100MB.wv, word2tfidf, embedding_dim) for text in corpus])
y_train = np.random.rand(len(corpus), 1)

# Defineix el model de Keras (ajusta les dimensions segons la longitud de les seqüències i la dimensió dels embeddings)
def build_and_compile_model(input_dim, hidden_size=64):
    input_1 = tf.keras.Input(shape=(input_dim,))
    input_2 = tf.keras.Input(shape=(input_dim,))
    
    concatenate = tf.keras.layers.Concatenate(axis=-1)([input_1, input_2])
    hidden = tf.keras.layers.Dense(hidden_size, activation='relu')(concatenate)
    output = tf.keras.layers.Dense(1)(hidden)
    
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)
    model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001))
    
    return model

# Construir i compilar el model
m = build_and_compile_model(input_dim=embedding_dim)

# Entrenar el model
m.fit([X_train1, X_train2], y_train, epochs=10, batch_size=32)

# Exemple de predicció
input_data1 = compute_mean_embedding("Aquest és un text de exemple".split(), model_100MB.wv, embedding_dim).reshape(1, -1)
input_data2 = compute_weighted_mean_embedding("Aquí hi ha un altre text".split(), model_100MB.wv, word2tfidf, embedding_dim).reshape(1, -1)

# Predir la similitud
y_pred = m.predict([input_data1, input_data2])
print(y_pred)

Epoch 1/10
1/1 [==============================] - 0s 334ms/step - loss: 0.7557
Epoch 2/10
1/1 [==============================] - 0s 3ms/step - loss: 0.1112
Epoch 3/10
1/1 [==============================] - 0s 2ms/step - loss: 0.4268
Epoch 4/10
1/1 [==============================] - 0s 5ms/step - loss: 0.5486
Epoch 5/10
1/1 [==============================] - 0s 3ms/step - loss: 0.4800
Epoch 6/10
1/1 [==============================] - 0s 5ms/step - loss: 0.3008
Epoch 7/10
1/1 [==============================] - 0s 3ms/step - loss: 0.0543
Epoch 8/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2087
Epoch 9/10
1/1 [==============================] - 0s 2ms/step - loss: 0.2880
Epoch 10/10
1/1 [==============================] - 0s 55ms/step
[[0.10263795]]


In [39]:
y_train

array([[0.24829134],
       [0.49559584]])